# Lab 5: Tools com MCP - Model Context Protocol

## 🎯 O que você vai aprender neste Lab

**MCP (Model Context Protocol)** é um padrão aberto que permite conectar agentes de IA a ferramentas e fontes de dados externas de forma padronizada.

### O que é MCP?

MCP cria uma interface padrão entre:
- **Agentes de IA** (clientes MCP)
- **Servidores de ferramentas** (servidores MCP)

```
┌─────────────────────────────────────────────────────┐
│                    ARQUITETURA MCP                   │
│                                                      │
│  ┌──────────┐    Padrão     ┌──────────────────┐   │
│  │  Agente  │ ◄──────────► │  Servidor MCP    │   │
│  │ LangChain│    Aberto     │  (ex: Time)      │   │
│  └──────────┘               └──────────────────┘   │
│       │                            │               │
│       ▼                            ▼               │
│  Recebe descrições          Executa operações      │
│  das tools                  reais                  │
└─────────────────────────────────────────────────────┘
```

### Vantagens do MCP:

1. **Padronização**: Uma interface para qualquer servidor de tools
2. **Reutilização**: Servidores MCP podem ser compartilhados
3. **Desacoplamento**: Agente e tools em processos separados
4. **Ecossistema**: Comunidade crescente de servidores MCP

> **Nota Databricks:** Este lab usa MCP que requer `npx` (Node.js). O MCP client pode não funcionar diretamente no Databricks devido a limitações de subprocess. Para testar MCP, execute localmente ou use uma alternativa como criar tools customizadas.

## ⚙️ Setup - Configuração do Ambiente

Instalação das bibliotecas, incluindo `langchain-mcp-adapters` para conexão com servidores MCP.

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community langchain-mcp-adapters nest_asyncio databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
import mlflow
mlflow.langchain.autolog()

## 🔌 Conectando a um Servidor MCP

Vamos conectar ao servidor `mcp-time` que fornece ferramentas relacionadas a tempo e fusos horários.

### O que o servidor `mcp-time` oferece:
- `current_time`: Hora atual em um fuso horário
- `convert_time_zone`: Converter entre fusos
- `add_time`: Adicionar duração a uma data
- `compare_time`: Comparar dois horários
- `relative_time`: Interpretar tempos relativos ("daqui a 2 horas")

In [0]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

## 🤖 Criando o Agente com Tools MCP

Agora criamos o agente passando as tools carregadas do servidor MCP. O processo é idêntico a tools locais!

In [ ]:
from langchain.agents import create_agent
from databricks_langchain import ChatDatabricks

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent_with_mcp = create_agent(
    model=llm,
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

## 🕐 Testando: Que Horas São em São Francisco?

Observe como o agente:
1. Recebe a pergunta
2. Identifica que precisa da tool `current_time`
3. Infere o fuso horário `America/Los_Angeles` de "SF"
4. Chama a tool no servidor MCP
5. Formata a resposta

In [0]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

## 📚 Resumo e Próximos Passos

### O que aprendemos:

| Conceito | Descrição |
|----------|-----------|
| **MCP** | Protocolo aberto para conectar agentes a servidores de tools |
| **MultiServerMCPClient** | Cliente LangChain para conectar a servidores MCP |
| **mcp_tools** | Tools carregadas do servidor, usadas como tools locais |
| **ainvoke** | Invocação assíncrona (necessária para comunicação com servidor) |

### Fluxo MCP vs Tools Locais:

```
Tools Locais:              MCP:
┌────────────────┐         ┌────────────────┐
│ @tool          │         │ Servidor MCP   │
│ def minha_tool │         │ (processo      │
│     ...        │         │  separado)     │
└────────────────┘         └────────────────┘
        │                          │
        ▼                          ▼
    Execução               Comunicação via
    no mesmo               protocolo padrão
    processo               
```

### Próximo Lab:
No **Lab 6 - Memory**, você aprenderá a adicionar memória ao agente para que ele lembre de conversas anteriores!